## Tutorial number 2 : Running a tractography pipeline on single shell data

This tutorial shows steps to run preprocessing and tractography (including freesurfer recon-all and connectome construction) on single shell data

### Define parameters

In [1]:
pipeline_parameters = {
    'freesurfer_path': '/usr/local/freesurfer/7.4.1',
    'eddyoptions_param': ' --slm=linear',
    'useants_param': True,
    'reconall_param': 'all',
    'fod_algorithm_param': 'tournier',
    'csd_algorithm_param': 'csd',
    'tt_algorithm_param': 'fsl',
    'flirt_interp_param': 'nearestneighbour',
    'flirt_dof_param': 8,
    'tckgen_algorithm_param': 'iFOD2',
    'tckgen_ntracks_param': 10000000,
    'tckgen_backtrack_param': True,
    'labelconvert_param': '/path/to/pytracto/pytracto/fs_a2009s.txt',
    'fs_lut_param': '/path/to/pytracto/pytracto/FreeSurferColorLUT.txt',
    'plugin_processing': 'MultiProc',
    'tckgen_method': ['Deterministic'],
    'atlas_list': ['Schaefer', 'Destrieux'],
}
run_steps = {
    'convert2bids': False,
    'run_pipeline': True,
    'createMatrixes': False,
    'createROIfile': False,
    'QAcheck': False,
    'bundleSegmentation': False,
    'ClusterConsensus': False,
}

general_parameters = {
    'session_list': ["01","02"],
    'source_dir': "/path/to/dir",
    'dicom_dir': '',
    'derivatives_folder': 'derivatives',
    'rawdata_folder': 'rawdata',
    'group': None,
    'group_list': None,
    'pipe_name': None,
    'result_name': None,
}
dim_template = {
    'anat' : [256,176,256],
    'dwi' : [[128,128,70,31],[128,128,70,7]]
} 


### Import modules

In [2]:
import os
import subprocess
from bids.layout import BIDSLayout
import time

from pytracto.BIDS_formatting.heudiconv_conhect import execute_bids_formatting

from pytracto.tractography.tractography_utils import *
from pytracto.tractography.odd_workflow import execute_odd_workflow
from pytracto.tractography.even_workflow import execute_even_workflow
from pytracto.tractography.synth_workflow import execute_synth_workflow
from pytracto.tractography.single_shell_workflow import execute_single_shell_workflow
from pytracto.tractography.single_shell_notopup import *

from pytracto.matrixescreation.MatrixesCreation import build_connectivity_matrixes
from pytracto.matrixescreation.createROIfile import create_roi_file

### Load parameters

In [3]:
source_dir = general_parameters.get("source_dir")
session_list = general_parameters.get("session_list")
rawdata_folder = general_parameters.get("rawdata_folder")
derivatives_folder = general_parameters.get("derivatives_folder")
group = general_parameters.get("group")
derivatives_dir = os.path.join(source_dir,derivatives_folder)
group_list = general_parameters.get("group_list")


if group:
	rawdata_dir = os.path.join(source_dir,rawdata_folder,group)
else:
	rawdata_dir = os.path.join(source_dir,rawdata_folder)


convert2bids = run_steps.get("convert2bids")
run_pipeline = run_steps.get("run_pipeline")
createMatrixes = run_steps.get("createMatrixes")
createROIfile = run_steps.get("createROIfile")
QAcheck = run_steps.get("QAcheck")
bundleSegmentation = run_steps.get("bundleSegmentation")
ClusterConsensus = run_steps.get("ClusterConsensus")
steps = [convert2bids,run_pipeline,createMatrixes,createROIfile,QAcheck,bundleSegmentation,ClusterConsensus]

### Run tractography on single shell workflows

In [4]:
if not os.path.isfile(os.path.join(rawdata_dir,"workflows.csv")):
	categories_repartition = automatic_repartition(rawdata_dir,dim_template)
else:
	categories_repartition = pd.read_csv(os.path.join(rawdata_dir,"workflows.csv"))
	
workflow_list = categories_repartition['category'].unique()

print(workflow_list)

if run_pipeline:
	for ses in session_list:
		ses_id = "ses-" + ses
		for wf_category in workflow_list:
			if wf_category == "singleshell_synth" or  wf_category == "single_shell":
				print(f"session {ses} : workflow : {wf_category}")

				subject_list  = categories_repartition[(categories_repartition['session_id'] == ses_id) & (categories_repartition['category'] == wf_category)]['subject_id']
				subject_list = [s[4:] for s in subject_list]

				print(subject_list)
				print(len(subject_list))
				
				wf_map.get(wf_category)(source_dir,rawdata_dir,derivatives_dir,subject_list,[ses],template,**pipeline_parameters)
		

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/dir/rawdata'